In [3]:
from logger import FileProcessLogger, file_line_count

In [4]:
def get_words_and_tags():
    tag_of_word = {}
    words_with_tag = {}
    filename = './dane/supertags.txt'
    with open(filename, 'r', encoding='utf-8') as file:
        logger = FileProcessLogger(filename)
        for line in file:
            logger.update()
            word, tag = line.lower().split()
            tag_of_word[word] = tag
            if tag in words_with_tag:
                words_with_tag[tag].append(word)
            else:
                words_with_tag[tag] = [word]
    return tag_of_word, words_with_tag

In [9]:
tag_of_word, words_with_tag = get_words_and_tags()

Processed 10% of the file contents...
Processed 20% of the file contents...
Processed 30% of the file contents...
Processed 40% of the file contents...
Processed 50% of the file contents...
Processed 60% of the file contents...
Processed 70% of the file contents...
Processed 80% of the file contents...
Processed 90% of the file contents...
Processed 100% of the file contents...


In [5]:
def get_bigram():
    filename = './dane/poleval_2grams.txt'
    bigram = {}
    with open(filename, 'r', encoding='utf-8') as file:
        logger = FileProcessLogger(filename)
        for line in file:
            logger.update()
            n, word1, word2 = line.lower().split()
            n = int(n)
            if n < 10: 
                continue
            if word1 in bigram:
                bigram[word1].append((word2, n))
            else:
                bigram[word1] = [(word2, n)]
    return bigram

In [6]:
bigram = get_bigram()

Processed 10% of the file contents...
Processed 20% of the file contents...
Processed 30% of the file contents...
Processed 40% of the file contents...
Processed 50% of the file contents...
Processed 60% of the file contents...
Processed 70% of the file contents...
Processed 80% of the file contents...
Processed 90% of the file contents...
Processed 100% of the file contents...


In [7]:
from collections import defaultdict
def to_unigram(bigram):
    unigram = {}
    unigram = defaultdict(lambda: 0, unigram)
    for word1 in bigram:
        for word2, n in bigram[word1]:
            unigram[word1] += n
            unigram[word2] += n
    return unigram

In [8]:
unigram = to_unigram(bigram)

In [113]:
import numpy as np


def get_unigram_probability(word):
    if word in unigram:
        return unigram[word]
    return 0.0001


def choose_from_unigram(words):
    probs = np.array([get_unigram_probability(x) for x in words])
    probs = probs / np.sum(probs)
    return str(np.random.choice(words, 1, p=probs)[0])


def get_tag(word):
    if word in tag_of_word:
        return tag_of_word[word]
    return tag_of_word[('^' + word)[-3:]]


def random_similar_sentence(original):
    words = original.lower().split()
    tags = list(map(get_tag, words))
    alternative_words = list(map(lambda tag: words_with_tag[tag], tags))
    chosen_words = list(map(choose_from_unigram, alternative_words))
    return ' '.join(chosen_words)

In [42]:
sentences = [
    "Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę",
    "Litwo Ojczyzno moja Ty jesteś jak zdrowie",
    "Jeden z pojmanych najemników probował odebrać to ostrze",
    "Jak może być inaczej dopóki gildia znajduje się poza imperialną kontrolą",
    "Jak sobie życzysz",
    "Książę popatrzył spode łba i zajął się sterami",
]

In [45]:
for sentence in sentences:
    print(random_similar_sentence(sentence))

ortograficzny poseł dostrzegł na niekonwencjonalnej głowie wówczas określoną prostytutkę
szyszko belo ta ty jesteś jak usamodzielnienie
jeden z podpisanych pracodawców los zaoferować to niebo
jak może być przykładowo oraz konstrukcja przedstawia się poza europejską jakością
jak sobie chcesz
książę odszedł spode dnia i przygotował się głosami


In [106]:
def get_bigram_probability(word, successor):
    if successor in bigram[word]:
        return bigram[word][successor]
    return 0.0001


def choose_from_bigram(word, successors):
    probs = np.array([get_bigram_probability(word, successor) for successor in successors])
    probs = probs / np.sum(probs)
    return str(np.random.choice(successors, 1, p=probs)[0])


def random_similar_sentence_v2(original):
    words = original.lower().split()
    tags = list(map(get_tag, words))
    alternative_words = list(map(lambda tag: words_with_tag[tag], tags))
    words_in_bigram = { word for word in bigram }
    
    i = 0
    sentence = []   
    while i < len(words):
        known_alternatives = set(alternative_words[i])
        available_words = known_alternatives.intersection(words_in_bigram)
        word = choose_from_unigram(list(available_words))
        sentence.append(word)
        i += 1
        
        while word in bigram and i < len(words):
            known_alternatives = set(alternative_words[i])
            available_words = known_alternatives.intersection(words_in_bigram)
            if len(available_words) == 0:
                break
            
            word = choose_from_bigram(word, list(available_words))
            sentence.append(word)
            i += 1
        else:
            sentence.append('|')
    return ' '.join(sentence[:-1])

In [70]:
for sentence in sentences:
    print(random_similar_sentence_v2(sentence))

konstytucyjny pasjonat powiedział po nieodżałowanej wściekliźnie gromko sygnowaną pociechę
anno graco moja ty pochodzisz jak drzewce
jeden ze spłacanych niemców rem przeszkolić tamto wyczekiwanie
jak może być okazale póki arabica uruchamia się poza brudną kunegundą
jak sobie wpadasz
książę zatriumfował spode globalizmu i podsłuchał się aspektami


In [93]:
s = 'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'
random_similar_sentence_v2(s)

'cały sielańczyk uwiecznił w niezawodowej eskorcie bezdzietnie wysuszoną istotę'

In [112]:
for _ in range(30):
    res = random_similar_sentence_v2(s)
    print(res)
    original_words = s.lower().split()
    res_words = res.split()
    for i in range(len(original_words)):
        wo = original_words[i]
        if wo not in tag_of_word:
            wo = wo[-3:]
        wr = res_words[i]
        if wr not in tag_of_word:
            wr = wr[-3:]
        to = tag_of_word[wo]
        tr = tag_of_word[wr]
        if to != tr:
            print(original_words[i], to, res_words[i], tr)

using unigram!!!!
prawny salwator orzekł nad niesławnej poduszce wówczas przerwaną skargę
using unigram!!!!
apelacyjny wicemarszałek popełnił pod niecałej marinie niepoprawnie lubianą audiencję
using unigram!!!!
jedyny namiestnik podszył na niedawnej nebrasce sejsmicznie wytworzoną przeprawę
using unigram!!!!
radziecki chmielak udostępnił między nieracjonalnej salamance etatowo wspominaną politykę
using unigram!!!!
członkowski jakub pokreślił nad nieorganicznej polance dwuetapowo wygłodzoną finlandię
using unigram!!!!
incydentalny rajca ugodził nad niezawodowej mieliźnie kanonicznie wyciąganą lawinę
using unigram!!!!
bankowy bartek odmienił nad niepodległej gamie treningowo zwartą komercjalizację
using unigram!!!!
amerykański bej zadzwonił w niezgodnej koszulce optymistycznie zaplanowaną subskrypcję
using unigram!!!!
przemysłowy ginter zawłaszczył pod niepożądanej skodzie niepotrzebnie zapinaną tamarę
using unigram!!!!
wiejski magister wczytał po niecodziennej metafizyce górnolotnie za